In [ ]:
import pandas as pd
import re

def lap_time_to_milliseconds(time_str):
    if pd.notna(time_str):
        # Regular expression pattern to match the time components
        time_pattern = r'(?:(\d+):)?(\d+):(\d+\.\d+)'
        match = re.match(time_pattern, time_str)

        if match:
            # Extract hours, minutes, seconds, and milliseconds
            hours, minutes, seconds = match.groups()
            if hours:
                total_time = int(hours) * 3600000 + int(minutes) * 60000 + float(seconds) * 1000
            else:
                total_time = int(minutes) * 60000 + float(seconds) * 1000

            return int(total_time)
    return None  # Return None for null values

# Example usage:
"""
lap_time = "1:38.109"
milliseconds = lap_time_to_milliseconds(lap_time)
print(milliseconds)  # Output: 98109
"""


In [35]:
# ----- QUALIFYING -----

# Convert qualifying times to milliseconds and add the new columns to the qualifying dataset
qualifying_dataset = pd.read_json("./f1db_json/qualifying.json")

qualifying_dataset["q1_ms"] = qualifying_dataset["q1"].apply(lambda x: lap_time_to_milliseconds(x))
qualifying_dataset["q2_ms"] = qualifying_dataset["q2"].apply(lambda x: lap_time_to_milliseconds(x))
qualifying_dataset["q3_ms"] = qualifying_dataset["q3"].apply(lambda x: lap_time_to_milliseconds(x))

# Convert all q1_ms, q2_ms, and q3_ms values to integers

qualifying_dataset["q1_ms"] = qualifying_dataset["q1_ms"].astype("Int64")
qualifying_dataset["q2_ms"] = qualifying_dataset["q2_ms"].astype("Int64")
qualifying_dataset["q3_ms"] = qualifying_dataset["q3_ms"].astype("Int64")

#print(qualifying_dataset.head())

# export to json with file suffix of milliseconds

qualifying_dataset.to_json("./f1db_json/qualifying_ms.json", orient="records")


In [36]:
# ----- RESULTS -----

results_dataset = pd.read_json("./f1db_json/results.json")

results_dataset["fastestLapTime_ms"] = results_dataset["fastestLapTime"].apply(lambda x: lap_time_to_milliseconds(x))

# convert fastestLapTime_ms to int

results_dataset["fastestLapTime_ms"] = results_dataset["fastestLapTime_ms"].astype('Int64')

#print(results_dataset.head())

# export to json with file suffix of milliseconds

results_dataset.to_json("./f1db_json/results_ms.json", orient="records")


In [37]:
# ----- SPRINT RESULTS -----

sresults_dataset = pd.read_json("./f1db_json/sprint_results.json")

sresults_dataset["fastestLapTime_ms"] = sresults_dataset["fastestLapTime"].apply(lambda x: lap_time_to_milliseconds(x))

# convert fastestLapTime_ms to int

sresults_dataset["fastestLapTime_ms"] = results_dataset["fastestLapTime_ms"].astype('Int64')

#print(sresults_dataset.head())

# export to json with file suffix of milliseconds

sresults_dataset.to_json("./f1db_json/sprint_results_ms.json", orient="records")


In [20]:
import pandas as pd

races_dataset = pd.read_json("./f1db_json/races.json", convert_dates=False)

def concat_date_time(date_str, time_str):
    if pd.notna(date_str):
        # If date_str is a Timestamp, extract the date part
        if isinstance(date_str, pd.Timestamp):
            date_str = date_str.date().isoformat()
        # If time_str is not available, use '00:00:00Z'
        if not pd.notna(time_str):
            time_str = '00:00:00'
        return f"{date_str}T{time_str}Z"
    else:
        return None

# Create a new column 'datetime' by combining 'date' and 'time'
races_dataset['datetime'] = races_dataset.apply(lambda row: concat_date_time(row['date'], row['time']), axis=1)

# Create similar columns for other fields (fp1, fp2, fp3, quali)
for session in ['fp1', 'fp2', 'fp3', 'quali', 'sprint']:
    date_col = f'{session}_date'
    time_col = f'{session}_time'
    datetime_col = f'{session}_datetime'
    races_dataset[datetime_col] = races_dataset.apply(lambda row: concat_date_time(row[date_col], row[time_col]), axis=1)

# Export to JSON with file suffix of 'datetime'
races_dataset.to_json("./f1db_json/races_datetime.json", orient="records")


In [39]:
# TODO: crossover pitstop time field with date from races.json to get pitstop datetime field working

races_dataset = pd.read_json("./f1db_json/races.json", convert_dates=False)
pitstops_dataset = pd.read_json("./f1db_json/pit_stops.json", convert_dates=False)

# Create a new column 'datetime' by combining 'date' from the corresponding race (raceId) and 'time' from pit_stops, using the concat_date_time function

def get_race_date(raceId):
    row = races_dataset[races_dataset['raceId'] == raceId]
    if not row.empty:
        return row['date'].values[0]
    else:
        return None

pitstops_dataset['datetime'] = pitstops_dataset.apply(lambda row: concat_date_time(get_race_date(row['raceId']), row['time']), axis=1)

# Export to JSON with file suffix of 'datetime'
pitstops_dataset.to_json("./f1db_json/pit_stops_datetime.json", orient="records")
